#### Load model

In [5]:
import pickle
import pandas as pd 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


filename = 'model.pkl'
load_model = pickle.load(open(filename,'rb'))
trip = pd.read_csv('tripadvisor_hotel_reviews.csv')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(trip['Review'])



# Let's try another one.This time we will take a positive review
pos_review = ["Rooms were old. Staff difficult to reach. Food bad. Loud room parties. Kid based place. No concierge room available. Pick another hotel. This is definitely on a low level with little interest in higher"]

# Tokenization
pos_review = tokenizer.texts_to_sequences(pos_review)

# padding
pos_review = pad_sequences(pos_review,maxlen=100,padding='post')

# prediction
review_predict = (load_model.predict(pos_review)>0.4).astype('int')

print(review_predict)

if review_predict[0] == 0:
    print("It's a negative review")
else:
    print("It's a positive review")\

print(load_model.predict(pos_review))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
[[0]]
It's a negative review
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[[0.34265783]]


#### Deploy: save as deploy.py

In [ ]:
import pickle
import pandas as pd 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from flask import Flask, render_template, request


app=Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))
trip = pd.read_csv('tripadvisor_hotel_reviews.csv')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(trip['Review'])



@app.route('/')
def home():
    result = ''
    return render_template('index.html', **locals())



@app.route('/predict', methods=['POST', 'GET'])
def predict():
    comment = request.form['comment']
    
    # Tokenization
    comment = tokenizer.texts_to_sequences(comment)
    
    # padding
    comment = pad_sequences(comment,maxlen=100,padding='post')
    
    # prediction
    review_predict = (model.predict(comment)>0.4).astype('int')
    
    if review_predict[0] == 1:
        result = "It's a negative review"
    else:
        result = "It's a positive review"


    return render_template('index.html', **locals())

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Aug/2024 12:47:48] "GET / HTTP/1.1" 200 -


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step


127.0.0.1 - - [20/Aug/2024 12:48:02] "POST /predict HTTP/1.1" 200 -


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


127.0.0.1 - - [20/Aug/2024 12:48:10] "POST /predict HTTP/1.1" 200 -


#### Deploy with sound: deploy.py

In [2]:
import pandas as pd 
import pickle
from flask import Flask, render_template, request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


app=Flask(__name__,  static_folder='static')
model = pickle.load(open('model.pkl', 'rb'))
trip = pd.read_csv('tripadvisor_hotel_reviews.csv')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(trip['Review'])



@app.route('/')
def home():
    result = ''
    return render_template('index.html', **locals())



@app.route('/predict', methods=['POST', 'GET'])
def predict():
    comment = request.form['comment']

    # Tokenization
    comment = tokenizer.texts_to_sequences(comment)

    # padding
    comment = pad_sequences(comment,maxlen=100,padding='post')

    # prediction
    review_predict = (model.predict(comment)>0.5).astype('int')

    if review_predict[0] == 1:
        result = "It's a negative review"
        audio_file = '0.mp3'
    else:
        result = "It's a positive review"
        audio_file = '1.mp3'

    return render_template('index.html', audio_file=audio_file)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Aug/2024 15:16:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2024 15:16:04] "GET /static/ HTTP/1.1" 404 -
127.0.0.1 - - [17/Aug/2024 15:16:04] "GET /favicon.ico HTTP/1.1" 404 -


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step


127.0.0.1 - - [17/Aug/2024 15:16:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2024 15:16:09] "GET /static/0.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [17/Aug/2024 15:16:09] "GET /favicon.ico HTTP/1.1" 404 -
